In [1]:
# Cell 1: Imports, global paths, and small utilities

from __future__ import annotations
import os, json, math, gc, time, warnings
from pathlib import Path
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import RobustScaler
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss
from sklearn.utils import shuffle as sk_shuffle
import joblib
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# --- Paths (Windows-friendly) ---
DATA_ROOT = Path(r"C:\engine_module_pipeline\DTC_stage\data\synthetic")
ARTIFACTS_ROOT = Path(r"C:\engine_module_pipeline\DTC_stage\artifacts")
ARTIFACTS_ROOT.mkdir(parents=True, exist_ok=True)

# --- Helper: load JSON ---
def read_json(p: Path) -> dict:
    with open(p, "r", encoding="utf-8") as f:
        return json.load(f)

# --- Helper: find valid file even if typo existed ---
def find_split_file(dtc_dir: Path, dtc_code: str, split: str) -> Path:
    expected = dtc_dir / f"{dtc_code}_synth_{split}.csv"
    if expected.exists():
        return expected
    # fallback for the common typo "vaild"
    if split == "valid":
        typo = dtc_dir / f"{dtc_code}_synth_vaild.csv"
        if typo.exists():
            return typo
    raise FileNotFoundError(f"Missing split file for {dtc_code}, split={split}. "
                            f"Tried: {expected} (and typo fallback for valid)")

# --- Small metric helpers ---
def safe_auc(y_true: np.ndarray, y_score: np.ndarray) -> float:
    # AUROC can fail if only one class present; return 0.5 (chance)
    try:
        return float(roc_auc_score(y_true, y_score))
    except Exception:
        return 0.5

def safe_auprc(y_true: np.ndarray, y_score: np.ndarray) -> float:
    try:
        return float(average_precision_score(y_true, y_score))
    except Exception:
        return np.mean(y_true)  # baseline prevalence

def reliability_bins(y_true: np.ndarray, y_prob: np.ndarray, n_bins: int = 10):
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    idx = np.digitize(y_prob, bins) - 1
    # collect per bin
    prob_mean, true_rate, count = [], [], []
    for b in range(n_bins):
        mask = idx == b
        if mask.any():
            prob_mean.append(y_prob[mask].mean())
            true_rate.append(y_true[mask].mean())
            count.append(mask.sum())
        else:
            prob_mean.append(np.nan)
            true_rate.append(np.nan)
            count.append(0)
    return np.array(prob_mean), np.array(true_rate), np.array(count)

# Repro
GLOBAL_SEED = 1337
np.random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)


In [2]:
# Cell 2: Per-DTC default window sizes and the dataset class

# Reasonable defaults per DTC family (seconds = samples when cadence=1s)
DEFAULT_WINDOW_BY_FAMILY = {
    "BOOST":   {"window": 48,  "stride": 1},   # P0234
    "MISFIRE": {"window": 64,  "stride": 1},   # P0300
    "ELECT":   {"window": 64,  "stride": 1},   # P0562
    "AIRFLOW": {"window": 64,  "stride": 1},   # P0101
    "CAT":     {"window": 128, "stride": 1},   # P0420
    "THERM":   {"window": 96,  "stride": 1},   # P0217, P0125
    "O2":      {"window": 64,  "stride": 1},   # P0133
    "SPEED":   {"window": 64,  "stride": 1},   # P0501
}

def dtc_family(dtc: str) -> str:
    if dtc == "P0234": return "BOOST"
    if dtc == "P0300": return "MISFIRE"
    if dtc == "P0562": return "ELECT"
    if dtc == "P0101": return "AIRFLOW"
    if dtc == "P0420": return "CAT"
    if dtc in ("P0217", "P0125"): return "THERM"
    if dtc == "P0133": return "O2"
    if dtc == "P0501": return "SPEED"
    # default
    return "AIRFLOW"

def get_default_window_stride(dtc: str) -> Tuple[int, int]:
    fam = dtc_family(dtc)
    spec = DEFAULT_WINDOW_BY_FAMILY.get(fam, {"window": 64, "stride": 1})
    return spec["window"], spec["stride"]

# --- Dataset class: builds sliding windows with per-timestep labels ---
class DtcWindowDataset(Dataset):
    def __init__(
        self,
        csv_path: Path,
        features: List[str],
        window: int,
        stride: int,
        fit_scaler: Optional[RobustScaler] = None,
        apply_scaler: Optional[RobustScaler] = None,
        cadence_seconds: float = 1.0,
    ):
        self.csv_path = csv_path
        self.features = features
        self.window = int(window)
        self.stride = int(stride)
        self.cadence_seconds = float(cadence_seconds)

        df = pd.read_csv(csv_path)
        # Ensure timestamp ordering
        if "timestamp" in df.columns:
            df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True, errors="coerce")
            df = df.sort_values("timestamp").reset_index(drop=True)

        # Impute features (forward fill then median)
        assert all(f in df.columns for f in features), f"Missing features in {csv_path}"
        X = df[features].copy()
        X = X.replace([np.inf, -np.inf], np.nan)
        X = X.fillna(method="ffill")
        X = X.fillna(X.median(numeric_only=True))

        # Scaler
        if fit_scaler is not None:
            fit_scaler.fit(X.values.astype(np.float32))
            self.scaler = fit_scaler
        else:
            self.scaler = apply_scaler
        Xs = self.scaler.transform(X.values.astype(np.float32)) if self.scaler is not None else X.values.astype(np.float32)

        # Labels per timestamp
        y_prec = df["y_precursor"].astype(np.float32).values
        y_fault = df["y_fault_active"].astype(np.float32).values

        # Build windows
        self.Xw, self.yw_prec, self.yw_fault = [], [], []
        self.t_end_idx = []  # index of window end in original array

        N = len(df)
        W = self.window
        for start in range(0, N - W + 1, self.stride):
            end = start + W
            self.Xw.append(Xs[start:end, :])
            self.yw_prec.append(y_prec[start:end])
            self.yw_fault.append(y_fault[start:end])
            self.t_end_idx.append(end - 1)

        self.Xw = np.stack(self.Xw).astype(np.float32)  # [B, W, C]
        self.yw_prec = np.stack(self.yw_prec).astype(np.float32)  # [B, W]
        self.yw_fault = np.stack(self.yw_fault).astype(np.float32)  # [B, W]
        self.timestamps = df["timestamp"] if "timestamp" in df.columns else None

    def __len__(self):
        return self.Xw.shape[0]

    def __getitem__(self, idx: int):
        x = self.Xw[idx]         # [W, C]
        yp = self.yw_prec[idx]   # [W]
        yf = self.yw_fault[idx]  # [W]
        return x, yp, yf, self.t_end_idx[idx]


In [3]:
# Cell 3: Causal TCN (two heads), loss, training/eval, and plotting

class Chomp1d(nn.Module):
    """Remove right padding to make conv effectively causal."""
    def __init__(self, chomp_size: int):
        super().__init__()
        self.chomp_size = chomp_size
    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous() if self.chomp_size > 0 else x

class TemporalBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, dilation, dropout=0.1):
        super().__init__()
        pad = (kernel_size - 1) * dilation
        self.net = nn.Sequential(
            nn.Conv1d(in_ch, out_ch, kernel_size, padding=pad, dilation=dilation),
            Chomp1d(pad),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Conv1d(out_ch, out_ch, kernel_size, padding=pad, dilation=dilation),
            Chomp1d(pad),
            nn.ReLU(),
            nn.Dropout(dropout),
        )
        self.downsample = nn.Conv1d(in_ch, out_ch, kernel_size=1) if in_ch != out_ch else None
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

class TCNTwoHead(nn.Module):
    """
    Input: x [B, T, C]
    Output: dict with:
        p_precursor: [B, T] probabilities
        p_fault:     [B, T] probabilities
    """
    def __init__(self, in_ch: int, hid: int = 64, n_blocks: int = 3, kernel_size: int = 3, dropout: float = 0.1):
        super().__init__()
        layers = []
        ch_in = in_ch
        for b in range(n_blocks):
            dilation = 2 ** b
            layers.append(TemporalBlock(ch_in, hid, kernel_size, dilation, dropout))
            ch_in = hid
        self.tcn = nn.Sequential(*layers)
        self.head_prec = nn.Conv1d(hid, 1, kernel_size=1)
        self.head_fault = nn.Conv1d(hid, 1, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # to [B, C, T] for conv1d
        x = x.transpose(1, 2)
        z = self.tcn(x)
        logit_prec = self.head_prec(z).squeeze(1)  # [B, T]
        logit_fault = self.head_fault(z).squeeze(1)
        p_prec = self.sigmoid(logit_prec)
        p_fault = self.sigmoid(logit_fault)
        return {"p_precursor": p_prec, "p_fault": p_fault, "logits": (logit_prec, logit_fault)}

def compute_pos_weight(y_seq: np.ndarray, eps: float = 1e-6) -> float:
    """
    y_seq: [N, T] 0/1
    """
    pos = y_seq.sum()
    neg = y_seq.size - pos
    return float((neg + eps) / (pos + eps))

def plot_loss_curves(train_hist: List[float], valid_hist: List[float], out_path: Path):
    plt.figure(figsize=(6,4))
    plt.plot(train_hist, label="train")
    plt.plot(valid_hist, label="valid")
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.title("Loss curves")
    plt.legend()
    plt.tight_layout()
    out_path.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(out_path.as_posix(), dpi=150)
    plt.close()

def plot_reliability(y, p, title, out_path: Path):
    pm, tr, cnt = reliability_bins(y, p, n_bins=10)
    plt.figure(figsize=(4.5,4.5))
    plt.plot([0,1],[0,1], "--", lw=1)
    mask = ~np.isnan(pm)
    plt.scatter(pm[mask], tr[mask], s=np.maximum(10, np.array(cnt[mask]) / np.max(cnt[mask]) * 80))
    plt.xlabel("Predicted probability (bin mean)")
    plt.ylabel("Empirical frequency")
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    out_path.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(out_path.as_posix(), dpi=150)
    plt.close()


In [4]:
# Cell 4 (FIXED): Training loop + evaluation + calibration + threshold tuning
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

def train_one_dtc(dtc_dir: Path) -> Dict:
    meta = read_json(dtc_dir / "meta.json")
    dtc_code: str = meta["dtc_code"]
    features: List[str] = meta["features"]
    cadence = float(meta.get("cadence_seconds", 1.0))
    win, stride = get_default_window_stride(dtc_code)

    # --- Split files ---
    f_train = find_split_file(dtc_dir, dtc_code, "train")
    f_valid = find_split_file(dtc_dir, dtc_code, "valid")
    f_test  = find_split_file(dtc_dir, dtc_code, "test")

    # --- Fit scaler on TRAIN only (IQR 25–75) ---
    scaler = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0))
    ds_train = DtcWindowDataset(f_train, features, window=win, stride=stride, fit_scaler=scaler, apply_scaler=None, cadence_seconds=cadence)
    ds_valid = DtcWindowDataset(f_valid, features, window=win, stride=stride, fit_scaler=None, apply_scaler=scaler, cadence_seconds=cadence)
    ds_test  = DtcWindowDataset(f_test,  features, window=win, stride=stride, fit_scaler=None, apply_scaler=scaler, cadence_seconds=cadence)

    # --- DataLoaders ---
    train_loader = DataLoader(ds_train, batch_size=64, shuffle=True, drop_last=True)
    valid_loader = DataLoader(ds_valid, batch_size=64, shuffle=False)
    test_loader  = DataLoader(ds_test,  batch_size=64, shuffle=False)

    # --- Model ---
    model = TCNTwoHead(in_ch=len(features), hid=64, n_blocks=3, kernel_size=3, dropout=0.10).to(DEVICE)
    # class imbalance
    pw_prec = compute_pos_weight(ds_train.yw_prec)
    pw_fault = compute_pos_weight(ds_train.yw_fault)
    loss_prec = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pw_prec, device=DEVICE))
    loss_fault = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pw_fault, device=DEVICE))

    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-3, weight_decay=1e-4)
    # FIX: use torch.optim.lr_scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=3, verbose=False)

    # --- Train ---
    best_state = None
    best_valid = 1e9
    train_hist, valid_hist = [], []
    EPOCHS = 20

    for epoch in range(1, EPOCHS + 1):
        model.train()
        tr_loss = 0.0
        for xb, ypb, yfb, _ in train_loader:
            xb = xb.to(DEVICE)                         # [B, T, C]
            ypb = ypb.to(DEVICE)                       # [B, T]
            yfb = yfb.to(DEVICE)
            out = model(xb)
            logit_prec, logit_fault = out["logits"]
            # per-timestep losses
            lp = loss_prec(logit_prec, ypb)
            lf = loss_fault(logit_fault, yfb)
            # TV smoothness penalty to reduce jitter (small)
            tv = (out["p_fault"][:, 1:] - out["p_fault"][:, :-1]).abs().mean() + \
                 (out["p_precursor"][:, 1:] - out["p_precursor"][:, :-1]).abs().mean()
            loss = lp + lf + 0.01 * tv
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
            optimizer.step()
            tr_loss += loss.item()
        tr_loss /= max(1, len(train_loader))
        train_hist.append(tr_loss)

        # validation loss
        model.eval()
        va_loss = 0.0
        with torch.no_grad():
            for xb, ypb, yfb, _ in valid_loader:
                xb = xb.to(DEVICE)
                ypb = ypb.to(DEVICE)
                yfb = yfb.to(DEVICE)
                out = model(xb)
                logit_prec, logit_fault = out["logits"]
                lp = loss_prec(logit_prec, ypb)
                lf = loss_fault(logit_fault, yfb)
                tv = (out["p_fault"][:, 1:] - out["p_fault"][:, :-1]).abs().mean() + \
                     (out["p_precursor"][:, 1:] - out["p_precursor"][:, :-1]).abs().mean()
                va_loss += (lp + lf + 0.01 * tv).item()
        va_loss /= max(1, len(valid_loader))
        valid_hist.append(va_loss)
        scheduler.step(va_loss)

        if va_loss < best_valid:
            best_valid = va_loss
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}

        print(f"[{dtc_code}] epoch {epoch:02d} | train={tr_loss:.4f} valid={va_loss:.4f} | pw(fault)={pw_fault:.2f}, pw(prec)={pw_prec:.2f}")

    # Restore best
    if best_state is not None:
        model.load_state_dict(best_state)

    # --- Evaluate on VALID (for calibration) + TEST (for final metrics) ---
    def collect_probs_labels(loader):
        model.eval()
        all_pp, all_pf, all_lp, all_lf = [], [], [], []
        with torch.no_grad():
            for xb, ypb, yfb, _ in loader:
                xb = xb.to(DEVICE)
                out = model(xb)
                all_pp.append(out["p_precursor"].cpu().numpy())
                all_pf.append(out["p_fault"].cpu().numpy())
                all_lp.append(ypb.numpy())
                all_lf.append(yfb.numpy())
        Pp = np.concatenate(all_pp, axis=0).reshape(-1)
        Pf = np.concatenate(all_pf, axis=0).reshape(-1)
        Lp = np.concatenate(all_lp, axis=0).reshape(-1)
        Lf = np.concatenate(all_lf, axis=0).reshape(-1)
        return Pp, Pf, Lp, Lf

    Pp_val, Pf_val, Lp_val, Lf_val = collect_probs_labels(valid_loader)
    Pp_tst, Pf_tst, Lp_tst, Lf_tst = collect_probs_labels(test_loader)

    # --- Calibration (isotonic) on VALID ---
    iso_prec = IsotonicRegression(out_of_bounds="clip")
    iso_fault = IsotonicRegression(out_of_bounds="clip")
    # Fit on raw probs from VALID
    iso_prec.fit(Pp_val, Lp_val)
    iso_fault.fit(Pf_val, Lf_val)
    # Calibrated
    Pp_val_c = iso_prec.transform(Pp_val)
    Pf_val_c = iso_fault.transform(Pf_val)
    Pp_tst_c = iso_prec.transform(Pp_tst)
    Pf_tst_c = iso_fault.transform(Pf_tst)

    # --- Metrics ---
    metrics = {
        "valid": {
            "AUROC_precursor": safe_auc(Lp_val, Pp_val_c),
            "AUPRC_precursor": safe_auprc(Lp_val, Pp_val_c),
            "Brier_precursor": float(brier_score_loss(Lp_val, Pp_val_c)),
            "AUROC_fault": safe_auc(Lf_val, Pf_val_c),
            "AUPRC_fault": safe_auprc(Lf_val, Pf_val_c),
            "Brier_fault": float(brier_score_loss(Lf_val, Pf_val_c)),
        },
        "test": {
            "AUROC_precursor": safe_auc(Lp_tst, Pp_tst_c),
            "AUPRC_precursor": safe_auprc(Lp_tst, Pp_tst_c),
            "Brier_precursor": float(brier_score_loss(Lp_tst, Pp_tst_c)),
            "AUROC_fault": safe_auc(Lf_tst, Pf_tst_c),
            "AUPRC_fault": safe_auprc(Lf_tst, Pf_tst_c),
            "Brier_fault": float(brier_score_loss(Lf_tst, Pf_tst_c)),
        }
    }

    # --- Threshold tuning (per-head) on VALID calibrated probs ---
    def tune_thresholds(y: np.ndarray, p: np.ndarray, cadence_s: float, head: str):
        dp = np.diff(p, prepend=p[:1]) / max(cadence_s, 1e-6)
        T_on_grid = np.linspace(0.5, 0.9, 9)
        T_off_delta = 0.10
        dPdt_grid = np.linspace(0.05, 0.20, 4)  # per second
        min_consec_grid = [1, 3, 5]
        best = None
        best_score = -1e9

        for T_on in T_on_grid:
            T_off = max(0.0, T_on - T_off_delta)
            for dthr in dPdt_grid:
                for mcons in min_consec_grid:
                    on = False
                    preds = np.zeros_like(p, dtype=np.uint8)
                    consec = 0
                    for i in range(len(p)):
                        trigger = (p[i] >= T_on) or (p[i] >= max(0.4, T_on - 0.1) and dp[i] >= dthr)
                        if on:
                            if p[i] <= T_off:
                                consec = 0
                                on = False
                        else:
                            if trigger:
                                consec += 1
                                if consec >= mcons:
                                    on = True
                                    consec = 0
                        preds[i] = 1 if on else 0
                    tp = int(((preds == 1) & (y == 1)).sum())
                    fp = int(((preds == 1) & (y == 0)).sum())
                    fn = int(((preds == 0) & (y == 1)).sum())
                    precision = tp / (tp + fp + 1e-9)
                    recall = tp / (tp + fn + 1e-9)
                    f1 = 2 * precision * recall / (precision + recall + 1e-9)
                    fpr = fp / max(1, int((y == 0).sum()))
                    score = f1 - 0.10 * fpr
                    if score > best_score:
                        best_score = score
                        best = {
                            "rate_threshold": float(dthr),
                            "min_consec_on": int(mcons),
                            "score": float(score),
                            "T_on": float(T_on),
                            "T_off": float(T_off),
                        }
        return best

    thr_prec = tune_thresholds(Lp_val, Pp_val_c, cadence, head="precursor")
    thr_fault = tune_thresholds(Lf_val, Pf_val_c, cadence, head="fault")

    thresholds = {
        "fault_on":  thr_fault["T_on"],
        "fault_off": thr_fault["T_off"],
        "prec_on":   thr_prec["T_on"],
        "prec_off":  thr_prec["T_off"],
        "dPdt_fault": thr_fault["rate_threshold"],
        "dPdt_prec":  thr_prec["rate_threshold"],
        "min_consec_on_fault": thr_fault["min_consec_on"],
        "min_consec_on_prec":  thr_prec["min_consec_on"],
        "cadence_seconds": cadence,
        "window_length": int(win),
        "stride": int(stride),
    }

    # --- Artifacts directory per DTC ---
    art_dir = ARTIFACTS_ROOT / dtc_code
    art_dir.mkdir(parents=True, exist_ok=True)

    # Save scaler
    joblib.dump(scaler, (art_dir / f"scaler_{dtc_code}.pkl").as_posix())

    # Save calibration models
    joblib.dump(iso_prec, (art_dir / f"calib_{dtc_code}_precursor.pkl").as_posix())
    joblib.dump(iso_fault, (art_dir / f"calib_{dtc_code}_fault.pkl").as_posix())

    # Save thresholds & feature spec
    feature_spec = {
        "dtc_code": dtc_code,
        "features": features,
        "window_length": int(win),
        "stride": int(stride),
        "cadence_seconds": cadence,
        "expects_masks": False,
    }
    (art_dir / "feature_spec.json").write_text(json.dumps(feature_spec, indent=2), encoding="utf-8")
    (art_dir / "thresholds.json").write_text(json.dumps(thresholds, indent=2), encoding="utf-8")

    # Save metrics + plots
    (art_dir / "metrics.json").write_text(json.dumps(metrics, indent=2), encoding="utf-8")
    plot_loss_curves(train_hist, valid_hist, art_dir / "loss_curves.png")
    plot_reliability(Lf_val, Pf_val_c, f"{dtc_code} fault reliability (valid)", art_dir / "calibration_fault_valid.png")
    plot_reliability(Lp_val, Pp_val_c, f"{dtc_code} precursor reliability (valid)", art_dir / "calibration_precursor_valid.png")

    # TorchScript export (model only; calibration & thresholds applied in runtime)
    scripted = torch.jit.script(model.cpu())
    scripted.save((art_dir / f"model_{dtc_code}.ts").as_posix())
    model.to(DEVICE)  # optional: move back if continuing

    # Return brief summary for registry
    return {
        "dtc_code": dtc_code,
        "artifacts_dir": art_dir.as_posix(),
        "model": f"model_{dtc_code}.ts",
        "scaler": f"scaler_{dtc_code}.pkl",
        "calibrators": {
            "precursor": f"calib_{dtc_code}_precursor.pkl",
            "fault": f"calib_{dtc_code}_fault.pkl",
        },
        "thresholds": "thresholds.json",
        "feature_spec": "feature_spec.json",
        "metrics": metrics,
    }


Using device: cpu


In [5]:
def find_all_dtc_dirs(data_root: Path) -> List[Path]:
    return sorted([p for p in data_root.glob("dtc_*") if p.is_dir()])

registry = {"artifacts_root": ARTIFACTS_ROOT.as_posix(), "dtcs": {}}
dtc_dirs = find_all_dtc_dirs(DATA_ROOT)
dtc_dirs = dtc_dirs[5:]
print(f"Found {dtc_dirs} DTC folders.")

Found [WindowsPath('C:/engine_module_pipeline/DTC_stage/data/synthetic/dtc_P0234'), WindowsPath('C:/engine_module_pipeline/DTC_stage/data/synthetic/dtc_P0300'), WindowsPath('C:/engine_module_pipeline/DTC_stage/data/synthetic/dtc_P0420'), WindowsPath('C:/engine_module_pipeline/DTC_stage/data/synthetic/dtc_P0501'), WindowsPath('C:/engine_module_pipeline/DTC_stage/data/synthetic/dtc_P0562')] DTC folders.


In [6]:
for dtc_dir in dtc_dirs:
    meta = read_json(dtc_dir / "meta.json")
    dtc_code = meta["dtc_code"]
    print(f"\n=== Training {dtc_code} ===")
    summary = train_one_dtc(dtc_dir)
    registry["dtcs"][dtc_code] = {
        "path": summary["artifacts_dir"],
        "model": summary["model"],
        "scaler": summary["scaler"],
        "calibrators": summary["calibrators"],
        "thresholds": summary["thresholds"],
        "feature_spec": summary["feature_spec"],
        "metrics": summary["metrics"]["test"],  # stash test metrics for quick view
    }
    gc.collect()

(ARTIFACTS_ROOT / "registry.json").write_text(json.dumps(registry, indent=2), encoding="utf-8")
print("\n✅ Training complete. Global registry at:", (ARTIFACTS_ROOT / "registry.json").as_posix())



=== Training P0234 ===
[P0234] epoch 01 | train=1.2309 valid=0.9005 | pw(fault)=3.69, pw(prec)=6.30
[P0234] epoch 02 | train=0.6029 valid=1.2268 | pw(fault)=3.69, pw(prec)=6.30
[P0234] epoch 03 | train=0.4128 valid=1.7350 | pw(fault)=3.69, pw(prec)=6.30
[P0234] epoch 04 | train=0.3044 valid=1.9057 | pw(fault)=3.69, pw(prec)=6.30
[P0234] epoch 05 | train=0.2436 valid=1.8055 | pw(fault)=3.69, pw(prec)=6.30
[P0234] epoch 06 | train=0.1714 valid=1.9906 | pw(fault)=3.69, pw(prec)=6.30
[P0234] epoch 07 | train=0.1523 valid=2.0057 | pw(fault)=3.69, pw(prec)=6.30
[P0234] epoch 08 | train=0.1412 valid=2.0291 | pw(fault)=3.69, pw(prec)=6.30
[P0234] epoch 09 | train=0.1309 valid=2.1089 | pw(fault)=3.69, pw(prec)=6.30
[P0234] epoch 10 | train=0.1100 valid=2.4233 | pw(fault)=3.69, pw(prec)=6.30
[P0234] epoch 11 | train=0.1062 valid=2.4968 | pw(fault)=3.69, pw(prec)=6.30
[P0234] epoch 12 | train=0.1020 valid=2.4673 | pw(fault)=3.69, pw(prec)=6.30
[P0234] epoch 13 | train=0.1000 valid=2.4309 | pw(fa